In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, layers
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation
import matplotlib.pyplot as plt

C:\Users\USER\anaconda3\envs\ai\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [3]:
Data = np.load('D:/Dropbox/AICleansing_ver2/2.ProcessedData/train_mimic_orgscale.npy')
# TrData = np.load('D:/Dropbox/AICleansing_ver2/2.ProcessedData/train_mimic_orgscale.npy')
# ValData = np.load('D:/Dropbox/AICleansing_ver2/2.ProcessedData/valid_vitaldb_orgscale.npy')

In [4]:
np.random.shuffle(Data)
split_idx = len(Data)//5

In [5]:
split_idx

70844

In [6]:
TrData = Data[split_idx:]
ValData = Data[:split_idx]

In [7]:
TrData.shape, ValData.shape

((283380, 3000), (70844, 3000))

In [8]:
#Ver2

TrDataFrame = tf.signal.frame(TrData.astype('float32'), 50, 50).numpy()
ValDataFrame = tf.signal.frame(ValData.astype('float32'), 50, 50).numpy()

# np.random.shuffle(TrDataFrame)
# np.random.shuffle(ValDataFrame)

TrMask = np.random.choice([0,1], size=TrDataFrame.shape, p=[0.1,0.9])
ValMask = np.random.choice([0,1], size=ValDataFrame.shape, p=[0.1,0.9])

Tr_X = TrDataFrame.copy()
Tr_Y = (TrDataFrame - 20.0) / (220.0 - 20.0)
# 평균 80, 표준 편차 25인 정규 분포에서 값을 뽑아서 채울 배열 생성
random_values = np.random.normal(loc=80, scale=25, size=TrDataFrame.shape)
# TrMask가 1인 위치에 뽑은 값을 할당
Tr_X[TrMask == 0] = random_values[TrMask == 0]
Tr_X = (Tr_X - 20.0) / (220.0 - 20.0)

TrMask[:,-10:,:] = 0
Tr_X[:,-10:,:] = Tr_X[:,-10:,:] + np.random.normal(loc=0.0, scale=0.05, size=Tr_X[:,-10:,:].shape)

Tr_X = np.clip(Tr_X, 0.0, 1.0)

Val_X = ValDataFrame.copy()
Val_Y = (ValDataFrame - 20.0) / (220.0 - 20.0)
# 평균 80, 표준 편차 25인 정규 분포에서 값을 뽑아서 채울 배열 생성
random_values = np.random.normal(loc=80, scale=25, size=ValDataFrame.shape)
# TrMask가 1인 위치에 뽑은 값을 할당
Val_X[ValMask == 0] = random_values[ValMask == 0]
Val_X = (Val_X - 20.0) / (220.0 - 20.0)

ValMask[:,-10:,:] = 0
Val_X[:,-10:,:] = Val_X[:,-10:,:] + np.random.normal(loc=0.0, scale=0.05, size=Val_X[:,-10:,:].shape)

Val_X = np.clip(Val_X, 0.0, 1.0)

del Data
del random_values
del TrDataFrame
del ValDataFrame

In [9]:
Tr_X.shape

(283380, 60, 50)

In [10]:
import numpy as np

from collections import namedtuple
import tensorflow as tf
from tensorflow_probability import distributions as tfd
from tensorflow.keras import layers, initializers, regularizers, constraints, activations

CONSTANT_INIT = initializers.Constant(0.05)
InterpolateInput = namedtuple("InterpolateInput", ["values", "mask", "times"])
InterpolateState = namedtuple("InterpolateState", ["x_keep", "t_keep"])

In [11]:
def replace_column(tensor, replacement, column_idx):
    left = tensor[:, :column_idx]
    right = tensor[:, column_idx+1:]
    new_tensor = tf.concat([left, tf.expand_dims(replacement, axis=1), right], axis=1)
    return new_tensor

def compute_last_observed_and_mean(x, mask):
    """
    Args:
    - x: A tensor of shape [batch_size, time_len, feature_dim]
    - mask: A tensor of shape [batch_size, time_len, feature_dim]
    
    Returns:
    - last_observed: A tensor representing the last observed value for each feature over time.
    - mean: A tensor representing the mean value for each feature over time.
    """

    # x shape: [batch_size, time_len, feature_dim]
    # mask shape: [batch_size, time_len, feature_dim]
    # print('clom-1',x)
    # print('clom-2',mask)

    # Initialize last_observed tensor with zeros
    last_observed = tf.zeros_like(x)
    # last_observed shape: [batch_size, time_len, feature_dim]
    # print('clom-3',last_observed)

    # Initialize last_values tensor with zeros
    dynamic_shape = tf.shape(x)
    last_values = tf.zeros([dynamic_shape[0], OrigDim])

    # last_values shape: [batch_size, feature_dim]
    # print('clom-4',last_values)

    # Iteratively compute the last observed values
    for t in range(x.shape[1]):
        last_values = mask[:, t] * x[:, t] + (1 - mask[:, t]) * last_values
        last_observed = replace_column(last_observed, last_values, t)
    # last_observed shape: [batch_size, time_len, feature_dim]
    # print('clom-5',last_observed)

    # Compute mean across the time axis and account for the mask
    mean = tf.math.reduce_sum(x, axis=1) / tf.math.reduce_sum(mask, axis=1)
    # print('clom-6',mean)
    # mean shape: [batch_size, feature_dim]

    mean = tf.expand_dims(mean, axis=1)
    # mean shape: [batch_size, 1, feature_dim]
    # print('clom-7',mean)

    mean = tf.tile(mean, [1, x.shape[1], 1])
    # mean shape: [batch_size, time_len, feature_dim]
    # print('clom-8',mean)

    return last_observed, mean

def compute_delta_t(s):
    """
    Compute the time differences between consecutive missing values based on the mask s.
    
    Args:
    - s: A tensor of shape [batch_size, time_len, feature_dim] containing binary values (1 for observed and 0 for missing).
    
    Returns:
    A tensor of shape [batch_size, time_len, feature_dim] containing time differences between consecutive missing values.
    """
    # s shape: [batch_size, time_len, feature_dim]

    # Find positions of missing values
    missing_positions = tf.where(s == 0)
    # missing_positions shape: [num_missing, 3] where num_missing is the number of zeros in s

    # Compute cumulative sum of s and use difference to get distances
    cumsum_s = tf.cumsum(s, axis=1)
    # cumsum_s shape: [batch_size, time_len, feature_dim]

    zeros_tensor = tf.zeros((tf.shape(s)[0], 1, tf.shape(s)[2]), dtype=s.dtype)
    # zeros_tensor shape: [batch_size, 1, feature_dim]

    distances = tf.concat([zeros_tensor, cumsum_s[:, :-1]], axis=1)
    # distances shape after concat: [batch_size, time_len, feature_dim]

    distances = cumsum_s - distances
    # distances shape: [batch_size, time_len, feature_dim]

    # Create a delta_t tensor initialized with distances for missing values
    delta_t = tf.where(s == 0, distances, 0)
    # delta_t shape: [batch_size, time_len, feature_dim]

    # For initial missing values, fill with their index + 1
    initial_missing = tf.math.cumprod(s, axis=1) == 0
    # initial_missing shape: [batch_size, time_len, feature_dim]

    arange_tensor = tf.reshape(tf.range(1, tf.shape(s)[1] + 1, dtype=s.dtype), (1, -1, 1))
    # arange_tensor shape after reshape: [1, time_len, 1]

    arange_tensor = tf.broadcast_to(arange_tensor, tf.shape(s))
    # arange_tensor shape after broadcast: [batch_size, time_len, feature_dim]

    delta_t = tf.where(initial_missing, arange_tensor, delta_t)
    # delta_t shape: [batch_size, time_len, feature_dim]
    
    return delta_t

In [12]:
__all__ = [
    "GRUDCell",
    "GRUD",
    "GRUDInput",
    "GRUDState",
]

CONSTANT_INIT = initializers.Constant(0.05)
IMPUTATION_TYPES = ["miwae", "decay", "forward", "zero", "raw"]

GRUDInput = namedtuple("GRUDInput", ["values", "mask", "times"])
GRUDState = namedtuple("GRUDState", ["h", "x_keep", "s_prev"])


def exp_relu(x):
    return K.exp(-K.relu(x))

def exp_softplus(x):
    return K.exp(-K.softplus(x))


class GRUDCell(layers.Layer):
    def __init__(
        self,
        units,
        x_imputation="decay",
        input_decay="exp_relu",
        hidden_decay="exp_relu",
        masking_decay=None,
        activation="tanh",
        recurrent_activation="hard_sigmoid",
        use_bias=True,
        use_decay_bias=True,
        feed_masking=True,
        kernel_initializer="glorot_uniform",
        recurrent_initializer="orthogonal",
        bias_initializer="zeros",
        decay_initializer=CONSTANT_INIT,
        kernel_regularizer=None,
        recurrent_regularizer=None,
        bias_regularizer=None,
        decay_regularizer=None,
        kernel_constraint=None,
        recurrent_constraint=None,
        bias_constraint=None,
        decay_constraint=None,
        dropout=0.,
        recurrent_dropout=0.,
        reset_after=False,
        **kwargs,
    ):

        if units < 0:
            raise ValueError(
                f"Received an invalid value for argument `units`, expected a positive integer, got {units}."
            )

        if x_imputation not in IMPUTATION_TYPES:
            raise ValueError(f"Unsupported imputation method: {x_imputation}")

        if kwargs.pop("implementation", 1) != 1:
            raise ValueError(f"GRUDCell only supports implementation=1")

        if reset_after is True:  # TODO: Check if this is correct.
            raise ValueError("GRUDCell only support reset_after=False")

        # NOTE: Disable caching device for GRUDCell to avoid potential errors.
        # By default use cached variable under v2 mode, see b/143699808.
        # if tf.compat.v1.executing_eagerly_outside_functions():
        #     self._enable_caching_device = kwargs.pop("enable_caching_device", True)
        # else:
        #     self._enable_caching_device = kwargs.pop("enable_caching_device", False)

        super().__init__(**kwargs)

        self.x_imputation = x_imputation
        self.use_decay_bias = use_decay_bias
        self.feed_masking = feed_masking

        self.units = units
        self.decay_initializer = initializers.get(decay_initializer)
        self.decay_regularizer = regularizers.get(decay_regularizer)
        self.decay_constraint = constraints.get(decay_constraint)

        if x_imputation not in ["decay", "miwae"]:
            input_decay = None

        with tf.keras.utils.custom_object_scope({"exp_relu": exp_relu, "exp_softplus": exp_softplus}):
            self.input_decay = None if input_decay is None else activations.get(input_decay)
            self.hidden_decay = None if hidden_decay is None else activations.get(hidden_decay)

            if self.feed_masking:
                self.masking_decay = None if masking_decay is None else activations.get(masking_decay)
                self._masking_dropout_mask = None
            else:
                self.masking_decay = None

        self.units = units
        self.activation = activations.get(activation)
        self.recurrent_activation = activations.get(recurrent_activation)
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(recurrent_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(recurrent_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = min(1.0, max(0.0, dropout))
        self.recurrent_dropout = min(1.0, max(0.0, recurrent_dropout))

        self.reset_after = False
        self.output_size = self.units

        self._input_dim = None

    @property
    def state_size(self):
        return (self.units, self._input_dim, self._input_dim)

    def build(self, input_shape):
        super().build(input_shape)

        self._input_dim = input_dim = input_shape[0][-1]

        # NOTE: Disable caching device for GRUDCell to avoid potential errors.
        # default_caching_device = rnn_utils.caching_device(self)

        self.kernel = self.add_weight(
            shape=(input_dim, self.units * 3),
            name="kernel",
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            # caching_device=default_caching_device,
        )
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units * 3),
            name="recurrent_kernel",
            initializer=self.recurrent_initializer,
            regularizer=self.recurrent_regularizer,
            constraint=self.recurrent_constraint,
            # caching_device=default_caching_device,
        )

        if self.use_bias:
            self.bias = self.add_weight(
                shape=(3 * self.units,),
                name="bias",
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                # caching_device=default_caching_device,
            )
        else:
            self.bias = None

        if self.input_decay is not None:
            self.input_decay_kernel = self.add_weight(
                shape=(input_dim,),
                name="input_decay_kernel",
                initializer=self.decay_initializer,
                regularizer=self.decay_regularizer,
                constraint=self.decay_constraint,
                # caching_device=default_caching_device,
            )
            if self.use_decay_bias:
                self.input_decay_bias = self.add_weight(
                    shape=(input_dim,),
                    name="input_decay_bias",
                    initializer=self.bias_initializer,
                    regularizer=self.bias_regularizer,
                    constraint=self.bias_constraint,
                    # caching_device=default_caching_device,
                )
            else:
                self.input_decay_bias = None
        else:
            self.input_decay_kernel = None

        if self.hidden_decay is not None:
            self.hidden_decay_kernel = self.add_weight(
                shape=(self._input_dim, self.units),
                name="hidden_decay_kernel",
                initializer=self.decay_initializer,
                regularizer=self.decay_regularizer,
                constraint=self.decay_constraint,
                # caching_device=default_caching_device,
            )
            if self.use_decay_bias:
                self.hidden_decay_bias = self.add_weight(
                    shape=(self.units,),
                    name="hidden_decay_bias",
                    initializer=self.bias_initializer,
                    regularizer=self.bias_regularizer,
                    constraint=self.bias_constraint,
                    # caching_device=default_caching_device,
                )
            else:
                self.hidden_decay_bias = None
        else:
            self.hidden_decay_kernel = None

        if self.feed_masking:
            self.masking_kernel = self.add_weight(
                shape=(self._input_dim, self.units * 3),
                name="masking_kernel",
                initializer=self.kernel_initializer,
                regularizer=self.kernel_regularizer,
                constraint=self.kernel_constraint,
                # caching_device=default_caching_device,
            )
            if self.masking_decay is not None:
                self.masking_decay_kernel = self.add_weight(
                    shape=(self._input_dim,),
                    name="masking_decay_kernel",
                    initializer=self.decay_initializer,
                    regularizer=self.decay_regularizer,
                    constraint=self.decay_constraint,
                    # caching_device=default_caching_device,
                )
                if self.use_decay_bias:
                    self.masking_decay_bias = self.add_weight(
                        shape=(self._input_dim,),
                        name="masking_decay_bias",
                        initializer=self.bias_initializer,
                        regularizer=self.bias_regularizer,
                        constraint=self.bias_constraint,
                        # caching_device=default_caching_device,
                    )
                else:
                    self.masking_decay_bias = None
            else:
                self.masking_decay_kernel = None
        else:
            self.masking_kernel = None

        self.built = True

        
    def get_dropout_mask_for_cell(self, inputs, training, count=3):
        if training:
            masks = []
            for i in range(count):
                masks.append(tf.nn.dropout(tf.ones_like(inputs[i]), self.dropout))
            return masks
        else:
            return [tf.ones_like(inputs[i]) for i in range(count)]


    def get_recurrent_dropout_mask_for_cell(self, h_tm1, training, count=1):
        if training:
            return [tf.nn.dropout(tf.ones_like(h_tm1), self.recurrent_dropout) for _ in range(count)]
        else:
            return [tf.ones_like(h_tm1) for _ in range(count)]

    
    def call(self, inputs, states, training=None):
        # print('grudc-1',inputs)  # values:[None, feature_dim], mask:[None, feature_dim], times:[None,1]
        # print('grudc-2',states)  # [None, n_hidden], [None, feature_dim], [None, feature_dim] 
        input_x, input_m, input_t = inputs
        h_tm1, x_keep_tm1, t_prev_tm1 = states

        delta_t = input_t - t_prev_tm1
        # print('grudc-3',delta_t)  # [None, feature_dim]

        dp_mask = self.get_dropout_mask_for_cell(inputs, training, count=3)
        # print('grudc-4',dp_mask)  # [None, feature_dim], [None, feature_dim], [None, 1]
        rec_dp_mask = self.get_recurrent_dropout_mask_for_cell(h_tm1, training, count=3)
        # print('grudc-5',rec_dp_mask)  # [None, n_hidden], [None, n_hidden], [None, n_hidden]

        if self.feed_masking:
            if 0. < self.dropout < 1. and self._masking_dropout_mask is None:
                self._masking_dropout_mask = self._create_dropout_mask(
                    tf.ones_like(input_m, dtype=tf.float32), training=training, count=3,
                )

            m_dp_mask = self._masking_dropout_mask
            # print('m_dp_mask',m_dp_mask)  # None
        
        if self.input_decay is not None:
            gamma_di = delta_t * self.input_decay_kernel
            if self.use_decay_bias:
                gamma_di = K.bias_add(gamma_di, self.input_decay_bias)
            gamma_di = self.input_decay(gamma_di)
            # print('gamma_di',gamma_di)  # [None, feature_dim]

        if self.hidden_decay is not None:
            gamma_dh = K.dot(delta_t, self.hidden_decay_kernel)
            if self.use_decay_bias:
                gamma_dh = K.bias_add(gamma_dh, self.hidden_decay_bias)
            gamma_dh = self.hidden_decay(gamma_dh)
            # print('gamma_dh',gamma_dh)  # [None, n_hidden]

        if self.feed_masking and self.masking_decay is not None:
            gamma_dm = delta_t * self.masking_decay_kernel
            if self.use_decay_bias:
                gamma_dm = K.bias_add(gamma_dm, self.masking_decay_bias)
            gamma_dm = self.masking_decay(gamma_dm)
            # print('gamma_dm',gamma_dm)

        # weighted sum between decayed last observation and grown 0 (empirical mean)
#         x_keep_t = tf.where(input_m, input_x, x_keep_tm1)
        x_keep_t = tf.where(input_m > 0.5, input_x, x_keep_tm1)
        # print('grudc-6',x_keep_t)  # [None, feature_dim]
#         x_t = tf.where(input_m, input_x, gamma_di * x_keep_t)
        x_t = tf.where(input_m > 0.5, input_x, gamma_di * x_keep_t)
        # print('grudc-7',x_t)  # [None, feature_dim]

        if self.hidden_decay is not None:
            h_tm1d = gamma_dh * h_tm1
        else:
            h_tm1d = h_tm1
        # print('grudc-8',h_tm1d)  # [None, n_hidden]

        if self.feed_masking:
            m_t = 1. - tf.cast(input_m, tf.float32)
            if self.masking_decay is not None:
                m_t = gamma_dm * m_t
            # print('m_t',m_t)  # [None, feature_dim]

        x_z, x_r, x_h = x_t, x_t, x_t
        # print('pass1d')

        if 0. < self.dropout < 1.:
            x_z *= dp_mask[0]
            x_r *= dp_mask[1]
            x_h *= dp_mask[2]

            if self.feed_masking:
                m_z, m_r, m_h = m_t * m_dp_mask[0], m_t * m_dp_mask[1], m_t * m_dp_mask[2]
        else:
            if self.feed_masking:
                m_z, m_r, m_h = m_t, m_t, m_t

        h_tm1_z, h_tm1_r = h_tm1d, h_tm1d
        # print('pass2d')

        if 0. < self.recurrent_dropout < 1.:
            h_tm1_z *= rec_dp_mask[0]
            h_tm1_r *= rec_dp_mask[1]

        z_t = K.dot(x_z, self.kernel[:, : self.units])
        r_t = K.dot(x_r, self.kernel[:, self.units : self.units * 2])
        hh_t = K.dot(x_h, self.kernel[:, self.units * 2 :])

        z_t = z_t + K.dot(h_tm1_z, self.recurrent_kernel[:, : self.units])
        r_t = r_t + K.dot(h_tm1_r, self.recurrent_kernel[:, self.units : self.units * 2])
        # print('pass3d')

        if self.feed_masking:
            z_t += K.dot(m_z, self.masking_kernel[:, : self.units])
            r_t += K.dot(m_r, self.masking_kernel[:, self.units : self.units * 2])
            hh_t += K.dot(m_h, self.masking_kernel[:, self.units * 2 :])

        if self.use_bias:
            z_t = K.bias_add(z_t, self.bias[: self.units])
            r_t = K.bias_add(r_t, self.bias[self.units : self.units * 2])
            hh_t = K.bias_add(hh_t, self.bias[self.units * 2 :])

        z_t = self.recurrent_activation(z_t)
        r_t = self.recurrent_activation(r_t)
        # print('pass4d')

        if 0. < self.recurrent_dropout < 1.:
            h_tm1_h = r_t * h_tm1d * rec_dp_mask[2]
        else:
            h_tm1_h = r_t * h_tm1d

        hh_t = self.activation(hh_t + K.dot(h_tm1_h, self.recurrent_kernel[:, self.units * 2 :]))
        # print('pass5d')
        h_t = z_t * h_tm1 + (1 - z_t) * hh_t
        # print('pass6d')
#         t_prev_t = tf.where(input_m, K.tile(input_t, [1, self.state_size[-1]]), t_prev_tm1)
        t_prev_t = tf.where(input_m > 0.5, K.tile(input_t, [1, self.state_size[-1]]), t_prev_tm1)
        # print('pass7d')
        # print('grudc-9',h_t)  # [None, n_hidden]
        # print('grudc-10',x_keep_t)  # [None, feature_dim]
        # print('grudc-11',t_prev_t)  # [None, feature_dim]
        return h_t, (h_t, x_keep_t, t_prev_t)

    def reset_masking_dropout_mask(self):
        self._masking_dropout_mask = None

    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        dtype = dtype or self.dtype
        if inputs is None:
            return (
                tf.zeros([batch_size, self.units], dtype=dtype),
                tf.zeros([batch_size, self._input_dim], dtype=dtype),
                tf.zeros([batch_size, self._input_dim], dtype=dtype)
            )
        else:
            if self.go_backwards:
                initial_t = tf.reduce_max(inputs[2], axis=1, keepdims=True)
            else:
                initial_t = tf.expand_dims(inputs[2][:, 0], axis=1)

            input_dim = inputs[0].shape[-1]

            return (
                tf.zeros([batch_size, self.units], dtype=dtype),
                tf.zeros([batch_size, input_dim], dtype=dtype),
                tf.tile(initial_t, [1, input_dim])
            )


class GRUD(layers.RNN):
    def __init__(
        self,
        units,
        x_imputation="decay",
        input_decay="exp_relu",
        hidden_decay="exp_relu",
        masking_decay=None,
        activation="tanh",
        recurrent_activation="hard_sigmoid",
        use_bias=True,
        use_decay_bias=True,
        feed_masking=True,
        kernel_initializer="glorot_uniform",
        recurrent_initializer="orthogonal",
        bias_initializer="zeros",
        decay_initializer=CONSTANT_INIT,
        kernel_regularizer=None,
        recurrent_regularizer=None,
        bias_regularizer=None,
        decay_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        recurrent_constraint=None,
        bias_constraint=None,
        decay_constraint=None,
        dropout=0.,
        recurrent_dropout=0.,
        return_sequences=False,
        return_state=False,
        go_backwards=False,
        stateful=False,
        unroll=False,
        time_major=False,
        reset_after=False,
        **kwargs,
    ):

        cell = GRUDCell(
            units,
            activation=activation,
            recurrent_activation=recurrent_activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            recurrent_initializer=recurrent_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            recurrent_regularizer=recurrent_regularizer,
            bias_regularizer=bias_regularizer,
            kernel_constraint=kernel_constraint,
            recurrent_constraint=recurrent_constraint,
            bias_constraint=bias_constraint,
            dropout=dropout,
            recurrent_dropout=recurrent_dropout,
            reset_after=reset_after,
            x_imputation=x_imputation,
            input_decay=input_decay,
            hidden_decay=hidden_decay,
            use_decay_bias=use_decay_bias,
            feed_masking=feed_masking,
            masking_decay=masking_decay,
            decay_initializer=decay_initializer,
            decay_regularizer=decay_regularizer,
            decay_constraint=decay_constraint,
            dtype=kwargs.get("dtype"),
            trainable=kwargs.get("trainable", True),
        )
        super().__init__(
            cell,
            return_sequences=return_sequences,
            return_state=return_state,
            go_backwards=go_backwards,
            stateful=stateful,
            unroll=unroll,
            time_major=time_major,
            **kwargs,
        )
        self.activity_regularizer = regularizers.get(activity_regularizer)

    def call(self, inputs, mask=None, training=None, initial_state=None):
        # print('grud-1',inputs)  # [None, time_len, feature_dim]
        # print('grud-2',mask)  # None
        def reset_dropout_mask():
            self.cell.dropout_mask = None
            
        def reset_recurrent_dropout_mask():
            self.cell.recurrent_dropout_mask = None
            
        def reset_masking_dropout_mask():
            self.cell.masking_dropout_mask = None

        reset_dropout_mask()
        reset_recurrent_dropout_mask()
        reset_masking_dropout_mask()
        # print('pass1')
        
        # The input should be dense, padded with zeros. If a ragged input is fed
        # into the layer, it is padded and the row lengths are used for masking.
        if isinstance(inputs, tf.RaggedTensor):
            is_ragged_input = True
            inputs, row_lengths = inputs.to_tensor(), inputs.row_lengths()
        else:
            is_ragged_input = False
            row_lengths = None

        self._validate_args_if_ragged(is_ragged_input, mask)
        
        # GRU does not support constants. Ignore it during process.
        inputs, initial_state, _ = self._process_inputs(
            inputs, initial_state, None
        )
        
        # print('pass2')
        if isinstance(mask, list):
            mask = mask[0]
        # print('pass3')
        input_shape = K.int_shape(inputs[0])
        # print('grud-3',input_shape)  # (None, time_len, feature_dim)
        timesteps = input_shape[0] if self.time_major else input_shape[1]
        # print('grud-4',timesteps)  # time_len

        kwargs = {"training": training}
        # print('pass4')
        self._maybe_reset_cell_dropout_mask(self.cell)
        # print('pass5')

        def step(cell_inputs, cell_states):
            return self.cell(cell_inputs, cell_states, **kwargs)

        last_output, outputs, states = K.rnn(
            step,
            inputs,
            initial_state,
            constants=None,
            go_backwards=self.go_backwards,
            mask=mask,
            unroll=self.unroll,
            input_length=(row_lengths if row_lengths is not None else timesteps),
            time_major=self.time_major,
            zero_output_for_mask=self.zero_output_for_mask,
#             return_all_outputs=self.return_sequences,
        )
        # print('grud-5',last_output)  # (None, n_hidden)
        # print('grud-6',outputs)  # (None, time_len, n_hidden)
        # print('grud-7',states)  # (None,n_hidden), (None,feature_dim), (None,feature_dim)
        if self.stateful:
            updates = [self.states[0].assign(states[0])]
            self.add_update(updates)
        # print('pass6')
        if self.return_sequences:
            if is_ragged_input:
                output = tf.RaggedTensor.from_tensor(outputs, lengths=row_lengths)
            else:
                output = outputs
        else:
            output = last_output
        # print('grud-8',output)  # (None, time_len, n_hidden)
        if self.return_state:
            return [output] + list(states)
        else:
            return output


In [13]:
CONSTANT_INIT = initializers.Constant(0.05)
InterpolateInput = namedtuple("InterpolateInput", ["values", "mask", "times"])
InterpolateState = namedtuple("InterpolateState", ["x_keep", "t_keep"])


class DecayCell(layers.AbstractRNNCell):
    def __init__(
        self,
        use_bias=True,
        decay="exp_softplus",
        decay_initializer="zeros",
        bias_initializer=CONSTANT_INIT,
        decay_regularizer=None,
        bias_regularizer=None,
        decay_constraint=None,
        bias_constraint=None,
        **kwargs,
    ):

        super().__init__(**kwargs)

        self.use_bias = use_bias
        with tf.keras.utils.custom_object_scope({"exp_relu": exp_relu, "exp_softplus": exp_softplus}):
            self.decay = None if decay is None else activations.get(decay)

        self.decay_initializer = initializers.get(decay_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.decay_regularizer = regularizers.get(decay_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.decay_constraint = constraints.get(decay_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

    def build(self, input_shape):
        self._input_dim = input_shape[0][-1]

        self.decay_kernel = self.add_weight(
            shape=(self._input_dim,),
            name="decay_kernel",
            initializer=self.decay_initializer,
            regularizer=self.decay_regularizer,
            constraint=self.decay_constraint,
        )
        if self.use_bias:
            self.decay_bias = self.add_weight(
                shape=(self._input_dim,),
                name="decay_bias",
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
            )

        self.built = True

    def call(self, inputs, states, training=None):
        # print('dc-1',inputs)  # [None, feature_dim], [None, feature_dim], [None, 1]
        # print('dc-2',states)  # [None, feature_dim], [None, feature_dim]
        x_input, m_input, t_input = inputs
        x_last, t_last = states
        t_delta = t_input - t_last
        # print('dc-3',t_delta)  # [None, feature_dim]

        gamma_di = t_delta * self.decay_kernel
        # print('dc-4',gamma_di)  # [None, feature_dim]
        if self.use_bias:
            gamma_di = K.bias_add(gamma_di, self.decay_bias)
        gamma_di = self.decay(gamma_di)
        # print('dc-5',gamma_di)  # [None, feature_dim]

        m_input = tf.cast(m_input, tf.bool)
        # print('m',m_input)
        # print('x',x_input)
        # print('g',gamma_di)
        # print('x_l',x_last)
        x_t    = tf.where(m_input, x_input, gamma_di * x_last + (1 - gamma_di) * x_input)
        # print('dc-6',x_t)  # [None, feature_dim]
        # x_t    = tf.where(m_input, x_input, gamma_di * x_last)
        x_keep = tf.where(m_input, x_input, x_last)
        # print('dc-7',x_keep)  # [None, feature_dim]
        t_keep = tf.where(m_input, t_input, t_last)
        # print('dc-8',t_keep)  # [None, feature_dim]
        return x_t, InterpolateState(x_keep, t_keep)

    @property
    def state_size(self):
        return InterpolateState(x_keep=self._input_dim, t_keep=self._input_dim)

    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        if inputs is None:
            return InterpolateState(
                tf.zeros((batch_size, self._input_dim), dtype=dtype),
                tf.zeros((batch_size, self._input_dim), dtype=dtype),
            )
        else:
            return InterpolateState(
                tf.zeros((batch_size, self._input_dim), dtype=dtype),
                inputs.times[:, 0, :],
            )


class ForwardCell(layers.AbstractRNNCell):
    def build(self, input_shape):
        self._input_dim = input_shape[0][-1]
        self.built = True

    def call(self, inputs, states, training=None):
        x_input, m_input, t_input = inputs
        x_last, t_last = states

        x_keep = tf.where(m_input, x_input, x_last)
        t_keep = tf.where(m_input, t_input, t_last)
        state = InterpolateState(x_keep, t_keep)
        return state, state

    def state_size(self):
        return InterpolateState(x_keep=self._input_dim, t_keep=self._input_dim)

    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        if inputs is None:
            return InterpolateState(
                tf.zeros((batch_size, self._input_dim), dtype=dtype),
                tf.zeros((batch_size, self._input_dim), dtype=dtype),
            )
        else:
            return InterpolateState(
                tf.zeros((batch_size, self._input_dim), dtype=dtype),
                tf.reduce_max(inputs.times, axis=1) if self.go_backwards else inputs.times[:, 0, :],
            )


class LinearScan(layers.RNN):
    def __init__(
        self,
        unroll=False,
        go_backwards=False,
        **kwargs,
    ):

        cell = ForwardCell(
            dtype=kwargs.get("dtype"),
            trainable=kwargs.get("trainable", False),
        )

        super().__init__(
            cell,
            return_sequences=True,
            return_state=False,
            go_backwards=go_backwards,
            stateful=False,
            unroll=unroll,
            **kwargs,
        )


class DecayInterpolate(layers.RNN):
    def __init__(
        self,
        use_bias=True,
        decay="exp_relu",
        decay_initializer="zeros",
        bias_initializer=CONSTANT_INIT,
        decay_regularizer=None,
        bias_regularizer=None,
        decay_constraint=None,
        bias_constraint=None,
        unroll=False,
        **kwargs,
    ):

        cell = DecayCell(
            use_bias=use_bias,
            decay=decay,
            decay_initializer=decay_initializer,
            decay_regularizer=decay_regularizer,
            decay_constraint=decay_constraint,
            bias_initializer=bias_initializer,
            bias_regularizer=bias_regularizer,
            bias_constraint=bias_constraint,
            dtype=kwargs.get("dtype"),
            trainable=kwargs.get("trainable", True),
        )

        super().__init__(
            cell,
            return_sequences=True,
            return_state=False,
            go_backwards=False,
            stateful=False,
            unroll=unroll,
            **kwargs,
        )


class LinearInterpolate(layers.Layer):
    def __init__(
        self,
        unroll=False,
        **kwargs,
    ):
        super().__init__()

        self.forward_scan  = LinearScan(unroll=unroll, go_backwards=False, **kwargs)
        self.backward_scan = LinearScan(unroll=unroll, go_backwards=True,  **kwargs)

    def call(self, inputs, mask=None, training=None):
        forwards  = self.forward_scan(inputs, mask=mask, training=training)
        backwards = self.backward_scan(inputs, mask=mask, training=training)

        x_t, t = inputs.values, inputs.times
        x_last, t_last = forwards.x_keep, forwards.t_keep
        x_next = tf.reverse(backwards.x_keep, axis=[1])
        t_next = tf.reverse(backwards.t_keep, axis=[1])

        # Linear interpolation. See https://en.wikipedia.org/wiki/Linear_interpolation
        x_itp = (x_last * (t_next - t) + x_next * (t - t_last)) / (t_next - t_last)
        x_itp = tf.where(tf.math.is_nan(x_itp), 0., x_itp)
        x_itp = tf.where(tf.math.is_inf(x_itp), 0., x_itp)

        return tf.where(inputs.mask, x_t, x_itp)


In [14]:
class SupNotMIWAE(tf.keras.Model):
    def __init__(
        self, 
        n_hidden:int=128, 
        n_train_latents:int = 10, 
        n_train_samples:int=1, 
        observe_dropout=None, 
        return_sequences=True, 
        z_dim:int = 32,
        min_latent_sigma: float = 0.0001,
        min_sigma: float = 0.001,
    ):
        super(SupNotMIWAE, self).__init__()
        self.n_hidden = n_hidden
        self.feature_dim = None
        self.n_train_samples = n_train_samples
        self.n_train_latents = n_train_latents
        self.return_sequences = return_sequences
        self.z_dim = z_dim
        self.min_latent_sigma = min_latent_sigma
        self.min_sigma = min_sigma
        
        if observe_dropout is None:
            self.observe_dropout = tf.constant(0., dtype=tf.float32)
        else:
            self.observe_dropout = observe_dropout

        # Encoder with GRU-D
        self.encoder = GRUD(self.n_hidden, return_sequences=True, x_imputation="decay", feed_masking=True, name="encoder")
        min_softplus = lambda x: self.min_latent_sigma + (1 - self.min_latent_sigma) * tf.nn.softplus(x)
        self.encoder_mu    = layers.Dense(self.z_dim, activation=None, name="encoder/mu")
        self.encoder_sigma = layers.Dense(self.z_dim, activation=min_softplus, name="encoder/sigma")

        # Prior
        self.prior_gru = layers.GRU(self.z_dim, return_sequences=True,   name="prior/gru")
        self.prior_mu  = layers.Dense(self.z_dim, activation=None,         name="prior/mu")
        self.prior_std = layers.Dense(self.z_dim, activation=min_softplus, name="prior/std")
        
        # Decoder with GRU
        self.decoder = layers.GRU(self.n_hidden, return_sequences=True, name="decoder")
        min_softplus = lambda x: self.min_sigma + (1 - self.min_sigma) * tf.nn.softplus(x)
        self.decoder_mu    = layers.Dense(OrigDim, activation=None,         name="decoder/mu")
        self.decoder_sigma = layers.Dense(OrigDim, activation=min_softplus, name="decoder/sigma")
        
        # Impute
        self.interpolator = DecayInterpolate(name="interpolator")

    def encode(self, times, x_obs, missing_mask):
        # print('enc-1',times)  # [batch, time_len]
        # print('enc-2',x_obs)  # [batch, time_len, feature_dim]
        # print('enc-3',missing_mask)  # [batch, time_len, feature_dim]
        times = tf.expand_dims(times, axis=-1)
        # print('enc-4',times)  # [batch, time_len, 1]
    
        r = self.encoder(GRUDInput(x_obs, missing_mask, times))
        # print('enc-5',r)  # [batch, time_len, n_hidden]
        z_mu = self.encoder_mu(r)
        # print('enc-6',z_mu)  # [batch, time_len, z_dim]
        z_sigma = self.encoder_sigma(r)
        # print('enc-7',z_sigma)  # [batch, time_len, z_dim]
        
        q_z = tfd.Normal(loc=z_mu, scale=z_sigma)
        # print('enc-8',q_z)  # batch_shape=[?, time_len, z_dim]
        return q_z   
        
    def prior(self, z_samples):
        # print('pr-1',z_samples)  # [n_latents, batch, time_len, z_dim]
        shape = tf.shape(z_samples)
        # print('pr-2',shape)
        z_samples = tf.reshape(z_samples, shape=(-1, Length, self.z_dim))
        # print('pr-3',z_samples)  # [n_latents*batch, time_len, z_dim]
        r = self.prior_gru(z_samples[:,1:,:],initial_state=z_samples[:,0,:])
        # print('pr-4',r)  # [n_latents*batch, time_len-1, z_dim]
        r = tf.reshape(r, shape=(self.n_train_latents, -1, Length-1, self.z_dim))
        # print('pr-5',r)  # [n_latents, batch, time_len-1, z_dim]
        p_z_mu = self.prior_mu(r)
        # print('pr-6',p_z_mu)  # [n_latents, batch, time_len-1, z_dim]
        p_z_sigma = self.prior_std(r)
        # print('pr-7',p_z_sigma)  # [n_latents, batch, time_len-1, z_dim]
        p_z_mu    = tf.pad(p_z_mu,    [[0, 0], [0, 0], [1, 0], [0, 0]], constant_values=0)
        # print('pr-8',p_z_mu)  # [n_latents, batch, time_len, z_dim]
        p_z_sigma = tf.pad(p_z_sigma, [[0, 0], [0, 0], [1, 0], [0, 0]], constant_values=1)
        # print('pr-9',p_z_sigma)  # [n_latents, batch, time_len, z_dim]
        p_z = tfd.Normal(loc=p_z_mu, scale=p_z_sigma)
        # print('pr-10',p_z)  # batch_shape=[n_latents, ?, time_len, z_dim]
        
        return p_z

    def decode(self, times, z):
        # print('dec-1',times)  # [batch, time_len]
        # print('dec-2',z)  # [n_latents, batch, time_len, z_dim]
        shape = tf.shape(z)
        z = tf.reshape(z, shape=(-1, Length, self.z_dim))
        # print('dec-3',z)  # [n_latents*batch, time_len, z_dim]
        h = self.decoder(z)
        # print('dec-4',h)  # [n_latents*batch, time_len, n_hidden]
        h = tf.reshape(h, shape=(self.n_train_latents, -1, Length, self.n_hidden))
        # print('dec-5',h)  # [n_latents, batch, time_len, n_hidden]
        x_tilde_mu = self.decoder_mu(h)
        # print('dec-6',x_tilde_mu)  # [n_latents, batch, time_len, feature_dim]
        x_tilde_sigma = self.decoder_sigma(h)
        # print('dec-7',x_tilde_sigma)  # [n_latents, batch, time_len, feature_dim]
        p_x_tilde = tfd.Normal(loc=x_tilde_mu, scale=x_tilde_sigma)
        # print('dec-8',p_x_tilde)  # batch_shape=[n_latents, ?, time_len, feature_dim]
                
        return p_x_tilde

    def missing_model_func(self, x):
        return self.missing_model(x)

    def call(self, inputs, training=True):
        x_obs, missing_mask = inputs
        # print('main-1',x_obs)  # x_obs shape: [batch, time_len, feature_dim]
        # print('main-2',missing_mask)  # missing_mask shape: [batch, time_len, feature_dim]
        
        last_observed, mean = compute_last_observed_and_mean(x_obs, missing_mask)
        # print('main-3',last_observed)  # last_observed shape: [batch, time_len, feature_dim]
        # print('main-4',mean)  # mean shape: [batch, time_len]
        
        delta_t = compute_delta_t(missing_mask)
        # print('main-5',delta_t)  # delta_t shape: [batch, time_len, feature_dim]
        
        single_time_array = tf.range(0, x_obs.shape[1] * 0.5, 0.5, dtype=tf.float32)
        dynamic_shape = tf.shape(x_obs)
        times = tf.broadcast_to(single_time_array, [dynamic_shape[0], dynamic_shape[1]])
        # print('main-6',times)  # times shape: [batch, time_len]
        
        # Set feature_dim based on x_obs if not already set
        if self.feature_dim is None:
            self.feature_dim = x_obs.shape[-1]
            
        n_samples = self.n_train_samples
        n_latents = self.n_train_latents
        
        # === VAE ===

        # Encoder
        q_z = self.encode(times, x_obs, missing_mask)
        # print('main-7',q_z)  # batch_shape=[?, time_len, z_dim]
        
        # Latent
        z_samples = q_z.sample(n_latents)
        # print('main-8',z_samples)  # [n_latents, batch, time_len, z_dim]
        
        # Prior
        p_z = self.prior(z_samples)
        # print('main-9',p_z)  # batch_shape=[n_latents, ?, time_len, z_dim]
        
        # Decoder
        p_x_tilde = self.decode(times, z_samples)
        # print('main-10',p_x_tilde)  # batch_shape=[n_latents, ?, time_len, feature_dim]
        
        # Impute
                
        # log p(xᵒ | zₖ)
        log_p_x_obs_given_z = tf.reduce_sum(tf.where(
            tf.cast(missing_mask, tf.bool),
            p_x_tilde.log_prob(x_obs),
            0.
        ), axis=-1)
        # print('main-11',log_p_x_obs_given_z)  # [n_latents, batch, time_len]

        log_p_z = p_z.log_prob(z_samples)
        # print('main-12',log_p_z)  # [n_latents, batch, time_len, z_dim]
        log_q_z_given_x_obs = q_z.log_prob(z_samples)
        # print('main-13',log_q_z_given_x_obs)  # [n_latents, batch, time_len, z_dim]
        
        if self.return_sequences:
            log_p_x_obs_given_z = tf.cumsum(log_p_x_obs_given_z, axis=-1)
            log_p_z = tf.cumsum(log_p_z, axis=-1)
            log_q_z_given_x_obs = tf.cumsum(log_q_z_given_x_obs, axis=-1)
        else:
            log_p_x_obs_given_z = tf.reduce_sum(log_p_x_obs_given_z, axis=-1, keepdims=True)
            log_p_z = tf.reduce_sum(log_p_z, axis=-1, keepdims=True)
            log_q_z_given_x_obs = tf.reduce_sum(log_q_z_given_x_obs, axis=-1, keepdims=True)
        # print('main-14',log_p_x_obs_given_z)  # [n_latents, batch, time_len]
        # print('main-15',log_p_z)  # [n_latents, batch, time_len, z_dim]
        # print('main-16',log_q_z_given_x_obs)  # [n_latents, batch, time_len, z_dim]

        
        # Generate: xₖⱼᵐ ~ p(xₖᵐ | zₖ)
        x_tilde = self.generate(p_x_tilde, training=training)
        # print('main-17',x_tilde)  # [n_samples, n_latents, batch, time_len, feature_dim]

        # Dropout
        drop_mask, log_m = self.generate_observe_dropout_mask(tf.shape(x_tilde), missing_mask, training=training)
        # print('main-18',drop_mask)  # [None, None, None, time_len, feature_dim]
        # print('main-19',log_m)  # [n_samples, n_latents, batch, time_len]
        
        # print('main-check',drop_mask)

        # Impute
        x_impute = self.impute(times, x_obs, x_tilde, drop_mask)
        # print('main-20',x_impute)   # [batch, time_len, feature_dim]
        
#         # Reconstruction error = p(xᵒ | z)
#         recon_error = -tf.reduce_mean(log_p_x_obs_given_z)
#         # print('main-21',recon_error)
        
        return x_impute
    
    
    def generate(self, p_x_tilde, training=True):
        
        n_samples = self.n_train_samples

        # xₖⱼᵐ ~ p(xₖᵐ | zₖ)
        x_tilde = p_x_tilde.sample(n_samples)

        return x_tilde

    def generate_observe_dropout_mask(self, shape, missing_mask, training=True):
        missing_mask = tf.cast(missing_mask, tf.bool)
        if tf.reduce_any(self.observe_dropout > 0.) and training:
            p_m = tfd.Bernoulli(probs=(1. - self.observe_dropout))
            drop_mask = p_m.sample(shape if self.observe_dropout.ndim == 0 else shape[:-1])

            log_m = tf.reduce_sum(tf.where(
                missing_mask,
                p_m.log_prob(drop_mask),
                0.,
            ), axis=-1)

            drop_mask = tf.cast(drop_mask, dtype=bool) & missing_mask

        else:
            drop_mask = tf.broadcast_to(missing_mask, shape)

            log_m = tf.zeros(shape[:4])

        if self.return_sequences:
            log_m = tf.cumsum(log_m, axis=-1)
        else:
            log_m = tf.reduce_sum(log_m, axis=-1, keepdims=True)

        return drop_mask, log_m

    def impute(self, times, x_obs, x_tilde, missing_mask):
        # print('imp-1',times)  # [batch, time_len]
        # print('imp-2',x_obs)  # [batch, time_len, feature_dim]
        # print('imp-3',x_tilde)  # [n_samples, n_latents, batch, time_len, feature_dim]
        # print('imp-4',missing_mask)  # [batch, time_len, feature_dim]
        # Interpolate obs and combine with generated missing
        x_comb = tf.where(tf.cast(missing_mask,tf.bool), x_obs, x_tilde)
        # print('imp-5',x_comb)  # [n_samples, n_latents, batch, time_len, feature_dim]

        shape = tf.shape(x_comb)
        n_tiles = self.n_train_samples * self.n_train_latents
        
        times = tf.expand_dims(tf.tile(times, multiples=[n_tiles, 1]), axis=-1)
        # print('imp-6',times)  # [n_samples*n_latents*batch, time_len, 1]
        x_comb = tf.reshape(x_comb, shape=[-1, Length, OrigDim])
        # print('imp-7',x_comb)  # [n_samples*n_latents*batch, time_len, feature_dim]
        missing_mask = tf.reshape(missing_mask, shape=[n_tiles * shape[2], shape[3], shape[4]])
        # print('imp-8',missing_mask)  # [n_samples*n_latents*batch, time_len, feature_dim]
        # print('imp1:',x_comb)
        # print('imp2:',missing_mask)
        # print('imp3:',times)

        x_impute = self.interpolator(InterpolateInput(values=x_comb, mask=missing_mask, times=times))
        # print('imp-9',x_impute)  # [n_samples*n_latents*batch, time_len, feature_dim]
        x_impute = tf.reshape(x_impute, shape=[shape[0], shape[1], shape[2], shape[3], shape[4]])   
        # print('imp-10',x_impute)  # [n_samples, n_latents, batch, time_len, feature_dim]
        
        x_impute_mean = tf.reduce_mean(x_impute, axis=[0,1])
        # print('imp-11',x_impute_mean)

        return x_impute_mean
    

In [46]:
Length = Tr_X.shape[1]
OrigDim = Tr_X.shape[-1]
InpLayer = tf.keras.layers.Input((Length, OrigDim))
InpMask  = tf.keras.layers.Input((Length, OrigDim))
ImputeOut = SupNotMIWAE(n_train_latents=10, n_train_samples=1)([InpLayer, InpMask])

learning_rate = 0.0005  # replace this with your desired learning rate
decay = 1e-6
adam = tf.keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

SNM = Model([InpLayer,InpMask ], ImputeOut)
SNM.compile(loss='mse', optimizer = adam)

In [47]:
from tensorflow.keras.callbacks import ModelCheckpoint

# ModelCheckpoint 설정
checkpoint_filepath = './results/SNM_epoch{epoch:d}_loss{loss:.7f}_valloss{val_loss:.7f}.hdf5'  # 저장할 파일 경로 및 이름
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',  # 검증 데이터의 loss를 기준으로 함
    mode='min',  # loss의 최솟값을 모니터링
    save_best_only=True  # 가장 낮은 loss 값을 가질 때만 저장
)

In [48]:
SNM.load_weights('./results/SNM_epoch2975_loss0.0002969_valloss0.0003326.hdf5')

In [49]:
SNM.fit([Tr_X, TrMask], tf.convert_to_tensor(Tr_Y), validation_data=([Val_X, ValMask], tf.convert_to_tensor(Val_Y)), batch_size=1500, verbose=1, shuffle=True, epochs=10000, callbacks=[model_checkpoint_callback] )

Epoch 1/10000
189/189 [==============================] - 88s 448ms/step - loss: 5.5457e-04 - val_loss: 3.5835e-04
Epoch 2/10000
189/189 [==============================] - 83s 441ms/step - loss: 3.2028e-04 - val_loss: 3.4766e-04
Epoch 3/10000
189/189 [==============================] - 84s 443ms/step - loss: 3.1600e-04 - val_loss: 3.5524e-04
Epoch 4/10000
189/189 [==============================] - 84s 446ms/step - loss: 3.1319e-04 - val_loss: 3.3709e-04
Epoch 5/10000
189/189 [==============================] - 84s 443ms/step - loss: 3.1215e-04 - val_loss: 3.4948e-04
Epoch 6/10000
189/189 [==============================] - 83s 438ms/step - loss: 3.1486e-04 - val_loss: 3.4326e-04
Epoch 7/10000
189/189 [==============================] - 83s 439ms/step - loss: 3.1094e-04 - val_loss: 3.4374e-04
Epoch 8/10000
189/189 [==============================] - 83s 437ms/step - loss: 3.1599e-04 - val_loss: 3.5007e-04
Epoch 9/10000
189/189 [==============================] - 83s 441ms/step - loss: 3.1291e-

KeyboardInterrupt: 

In [17]:
# SNM.load_weights('./results/model_rightedge_mask_no_vital_val_2.hdf5')

In [18]:
# SNM.fit([Tr_X, TrMask], tf.convert_to_tensor(Tr_Y), validation_data=([Val_X, ValMask], tf.convert_to_tensor(Val_Y)), batch_size=3000, verbose=1, shuffle=True, epochs=10000, callbacks=[model_checkpoint_callback] )

Epoch 1/10000
95/95 [==============================] - 107s 1s/step - loss: 5.3819e-04 - val_loss: 5.3310e-04
Epoch 2/10000
95/95 [==============================] - 96s 1s/step - loss: 5.3851e-04 - val_loss: 5.3305e-04
Epoch 3/10000
95/95 [==============================] - 95s 1s/step - loss: 5.3596e-04 - val_loss: 5.3271e-04
Epoch 4/10000
95/95 [==============================] - 95s 1s/step - loss: 5.3761e-04 - val_loss: 5.3272e-04
Epoch 5/10000
95/95 [==============================] - 95s 1s/step - loss: 5.3589e-04 - val_loss: 5.3263e-04
Epoch 6/10000
95/95 [==============================] - 95s 1s/step - loss: 5.3716e-04 - val_loss: 5.3273e-04
Epoch 7/10000
95/95 [==============================] - 95s 1s/step - loss: 5.3462e-04 - val_loss: 5.3272e-04
Epoch 8/10000
95/95 [==============================] - 95s 1s/step - loss: 5.3485e-04 - val_loss: 5.3216e-04
Epoch 9/10000
95/95 [==============================] - 95s 1s/step - loss: 5.3494e-04 - val_loss: 5.3201e-04
Epoch 10/10000
95/

KeyboardInterrupt: 

In [ ]:
# SNM.load_weights('./results/model_rightedge_mask_no_vital_val.hdf5')

In [ ]:
# SNM.fit([Tr_X, TrMask], tf.convert_to_tensor(Tr_Y), validation_data=([Val_X, ValMask], tf.convert_to_tensor(Val_Y)), batch_size=3000, verbose=1, shuffle=True, epochs=10000, callbacks=[model_checkpoint_callback] )

In [ ]:
# SNM.fit([Tr_X, TrMask], tf.convert_to_tensor(Tr_Y), validation_data=([Val_X, ValMask], tf.convert_to_tensor(Val_Y)), batch_size=3000, verbose=1, shuffle=True, epochs=10000, callbacks=[model_checkpoint_callback] )

In [ ]:
SNM.summary()